In [13]:
import sys
sys.path.append("/path/to/codon_amino_cov/utils") #path to /codon_amino_cov/utils directory in repository
from Bio import SeqIO
import pandas as pd
import glob
import os
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score
import joblib

In [4]:
#Example: orf1ab codon model .joblib files
model_files = {os.path.basename(f).split("_")[0]:f for f in glob.glob("/path/to/codon_amino_rscu_final/orf1ab/codon/*joblib")}

In [5]:
meta = pd.read_table("/path/to/codon_amino_rscu_final/complete_cov_human_pathogen_meta.tsv")
#download model data from https://doi.org/10.5281/zenodo.14851561
model_data = pd.read_table("/path/to/codon_amino_rscu_final/complete_model_statistics.tsv")
#Example: load orf1ab data
seqs = {s.id:s for s in SeqIO.parse("/path/to/codon_amino_rscu_final/complete_cov_CDS_orf1ab.fasta",'fasta')} #CDS fasta file path for inference
meta = meta[meta['accession'].isin(set(seqs.keys()))].copy()
labels_dict = dict(meta[['accession','label']].values)

In [6]:
#Example: orf1ab codon frequency
#Model types include:logit__orf1ab_CDS, logit__nucleocapsid_CDS, and logit__spike_CDS
#Feature types include codon, rscu, and amino 
selected_models = model_data[(model_data['model_type'] == 'logit__orf1ab_CDS') & (model_data['feature_type'] == 'codon')].copy()

In [11]:
#classification results
resample = 200
results = []
for model in selected_models.iterrows():
    test_resample_index = np.zeros(resample)
    split = eval(model[1]['test_groups'])
    print("Test Split Group:", split)
    test_meta = meta[meta['groups'].isin(set(split))].copy()
    test_groups = test_meta.groupby('groups')
    print("Resampling {} times.".format(resample))
    for i in range(resample):
        class_pick = np.random.randint(2)
        selected = split[class_pick]
        test_resample_index[i] = test_groups.get_group(selected).sample(1).index[0]
    test_accessions = [a for a in test_meta.loc[test_resample_index]['accession']]
    X_test =  [str(seqs[a].seq) for a in test_accessions]
    y_test = np.array([labels_dict[a] for a in test_accessions])
    clf = joblib.load(model_files[model[1]['model_id']])
    print("Scoring results for model id:{}".format(model[1]['model_id']))
    #can replace accuracy with another scoring method such as recall_score
    results.append(accuracy_score(y_test,clf.predict(X_test)))

Test Split Group: ('1_11137', '0_694014')
Resampling 200 times.
Scoring results for model id:6793d38b409aa3e39a9b1a24
Test Split Group: ('1_1335626', '0_693999')
Resampling 200 times.
Scoring results for model id:6793d3d09ca8beae600c5d19
Test Split Group: ('1_290028', '0_1159906')
Resampling 200 times.
Scoring results for model id:6793ec42409aa3e39a9b1a25
Test Split Group: ('1_694003', '0_1739625')
Resampling 200 times.
Scoring results for model id:6793ed049ca8beae600c5d1a
Test Split Group: ('1_694003', '0_694003')
Resampling 200 times.
Scoring results for model id:679404ff409aa3e39a9b1a26
Test Split Group: ('1_694003', '0_1159902')
Resampling 200 times.
Scoring results for model id:679406c09ca8beae600c5d1b
Test Split Group: ('1_290028', '0_1159908')
Resampling 200 times.
Scoring results for model id:67941d68409aa3e39a9b1a27
Test Split Group: ('1_2697049', '0_1159903')
Resampling 200 times.
Scoring results for model id:67941fbc9ca8beae600c5d1c
Test Split Group: ('1_11137', '0_2501928')

Test Split Group: ('1_694009', '0_1335626')
Resampling 200 times.
Scoring results for model id:67975630444240fdf873c6fd
Test Split Group: ('1_11137', '0_694005')
Resampling 200 times.
Scoring results for model id:679765c4409aa3e39a9b1a49
Test Split Group: ('1_1335626', '0_1159902')
Resampling 200 times.
Scoring results for model id:67976ed7444240fdf873c6fe
Test Split Group: ('1_694003', '0_1335626')
Resampling 200 times.
Scoring results for model id:67977ec4409aa3e39a9b1a4a
Test Split Group: ('1_11137', '0_1965093')
Resampling 200 times.
Scoring results for model id:679787f5444240fdf873c6ff
Test Split Group: ('1_290028', '0_2032731')
Resampling 200 times.
Scoring results for model id:6797971a409aa3e39a9b1a4b
Test Split Group: ('1_290028', '0_694003')
Resampling 200 times.
Scoring results for model id:6797a09f444240fdf873c700
Test Split Group: ('1_11137', '0_694006')
Resampling 200 times.
Scoring results for model id:6797b01a409aa3e39a9b1a4c
Test Split Group: ('1_2697049', '0_693998')
R

In [12]:
np.mean(results)

0.77335